In [1]:
# GPU 사용 가능 여부 확인
import torch
print("GPU 사용 가능 여부:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))

GPU 사용 가능 여부: True
사용 중인 GPU: Tesla T4


In [2]:
# 필수 패키지 설치
!pip install -U sentence-transformers faiss-cpu scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# GitHub 레포 클론 (영화 수집 데이터 포함)
!git clone https://github.com/deyang0325/movie-reasoning-agent.git
%cd movie-reasoning-agent

Cloning into 'movie-reasoning-agent'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 40 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 13.99 MiB | 11.67 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/movie-reasoning-agent


In [4]:
# 디렉토리 확인
import os
print("현재 폴더 안의 파일 및 디렉토리:")
print(os.listdir())

print("\ndata_collector 폴더 내용:")
print(os.listdir("data_collector"))

현재 폴더 안의 파일 및 디렉토리:
['README.md', 'embedding_search_A_role.ipynb', 'movie_reasoning_llm_agent(ver_2).py', 'movie_reasoning_llm_agent.py', 'movie_metas.pkl', 'movie_index.faiss', 'data_collector', 'movie_sentences.pkl', '.git']

data_collector 폴더 내용:
['README.md', 'tmdb_korean_movies_2025.csv', 'tmdb_korean_movies_2021.csv', 'tmdb_korean_movies_2024.csv', 'tmdb_korean_movies_2023.csv', 'requirements.txt', 'tmdb_korean_movies_2011~2020.csv', 'collect_movie_tmdb.py', 'tmdb_korean_movies_2001~2010.csv', 'tmdb_korean_movies_2022.csv']


In [5]:
# 영화 수집 데이터 파일 로드 및 병합 (총 영화수 : 3,139개)
import pandas as pd

dfs = []
for year in range(2021, 2025):
    path = f"data_collector/tmdb_korean_movies_{year}.csv"
    df = pd.read_csv(path)
    df["year"] = year
    dfs.append(df)

movies_df = pd.concat(dfs, ignore_index=True)
print(f"총 영화 수: {len(movies_df)}")

총 영화 수: 3139


In [6]:
# 전처리 함수 정의 (줄거리+주연+장르+감독 병합)
# 개선된 search_text 생성 함수
def build_search_text(row):
    parts = []

    # 줄거리
    if pd.notna(row.get("줄거리")):
        parts.append(row["줄거리"].strip())

    # 주연, 감독, 장르를 강제로 포함시켜 인물/장르 검색 강화
    actor = row.get("주연", "")
    director = row.get("감독", "")
    genre = row.get("장르", "")

    if isinstance(actor, str) and actor:
        parts.append(actor)
    if isinstance(director, str) and director:
        parts.append(director)
    if isinstance(genre, str) and genre:
        parts.append(genre)

    return " ".join(parts)

In [7]:
# 줄거리 결측치 제거 및 검색용 텍스트 생성 (줄거리 포함 총 영화수 : 2,325)
filtered_df = movies_df.dropna(subset=["줄거리"]).copy().reset_index(drop=True)

# 검색용 텍스트 생성 함수 적용
filtered_df["search_text"] = filtered_df.apply(build_search_text, axis=1)

print("줄거리 포함 영화 수:", len(filtered_df))
print("예시 문장:", filtered_df['search_text'].iloc[0])

줄거리 포함 영화 수: 2325
예시 문장: 연애와 인간관계에도 후기를 남길 수 있다면...?  포스트 코로나 시대, 비대면 만남이 일상화된 생활 속 자만추보다는 데이팅 어플이 익숙해져버린 미래. '러브 플래닛'은 전 애인에 대한 솔직한 후기를 남길 수 있는 기능으로 사용자 3500만명을 보유하게 된 명실상부 대한민국 최대 데이팅 앱이다.  바로 이 러브 플래닛 고객 센터에서 일하는 '제시카'는 상담과 영업에 잔뼈가 굵은 프로 영업꾼.  평소처럼 전화 상담을 하던 어느 날, 그녀는 자기소개부터 심상치 않은 고객 '남자'를 상대하게 된다. 착한건지, 나쁜건지, 이상한건지 모르겠는 이 남자와의 어질어질한 밀당 영업... 그리고 충격적인 후기들. 오늘 정말 쉽지 않다!!! 황예정 SF, 로맨스


In [8]:
sentences = []
metas = []

for i, row in filtered_df.iterrows():
    plot = row.get('줄거리', '')
    actors = row.get('주연', '')
    genre = row.get('장르', '')
    director = row.get('감독', '')

    search_text = " ".join([
        f"줄거리: {plot}" if isinstance(plot, str) else "",
        f"출연: {actors}" if isinstance(actors, str) else "",
        f"장르: {genre}" if isinstance(genre, str) else "",
        f"감독: {director}" if isinstance(director, str) else "",
    ]).strip()

    if len(search_text) > 10:
        sentences.append(search_text)
        metas.append({
            "title": row.get("제목", ""),
            "genre": genre,
            "year": row.get("year", ""),
            "director": director,
            "cast": actors,
            "plot": plot
        })

In [9]:
# 문서 임베딩 및 FAISS 인덱스 생성 (결과 : 384차원)
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 한국어 특화 임베딩 모델로 교체
sentence_model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS", device="cuda")

# 문장 리스트 추출
search_texts = filtered_df["search_text"].tolist()

# 문장 임베딩
embeddings = sentence_model.encode(sentences, batch_size=32, show_progress_bar=True)

# FAISS 인덱스 생성 및 추가
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings).astype('float32'))

print(f"임베딩 완료 및 FAISS 인덱스 등록 ({dim}차원)")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/73 [00:00<?, ?it/s]

임베딩 완료 및 FAISS 인덱스 등록 (768차원)


In [10]:
# 메타데이터 구성
metas = filtered_df[["제목", "장르", "줄거리", "주연", "감독", "year"]].rename(
    columns={
        "제목": "title",
        "장르": "genre",
        "줄거리": "plot",
        "주연": "actors",
        "감독": "director"
    }
).to_dict(orient="records")

In [11]:
# 무결성 확인
assert len(sentences) == len(metas), "sentences와 metas 길이가 다릅니다!"

# Google Drive 마운트 및 저장
from google.colab import drive
import pickle
import os

drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/Colab Notebooks"
os.makedirs(save_path, exist_ok=True)

# 1. FAISS 인덱스 저장
faiss.write_index(index, os.path.join(save_path, "movie_index.faiss"))

# 2. 문장 리스트 저장 (search_texts or sentences → 사용한 변수로!)
with open(os.path.join(save_path, "movie_sentences.pkl"), "wb") as f:
    pickle.dump(search_texts, f)

# 3. 메타 정보 저장
with open(os.path.join(save_path, "movie_metas.pkl"), "wb") as f:
    pickle.dump(metas, f)

print("FAISS 인덱스 및 메타데이터 저장 완료!")

Mounted at /content/drive
FAISS 인덱스 및 메타데이터 저장 완료!
